In [1]:
import data_preprocessing.proto_interface.data_for_learning_pb2
from data_preprocessing.proto_interface.data_for_learning_pb2 import *

import numpy as np

import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import torch

from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence

In [2]:
def LoadDataForLearning(filepath):
    list_of_data_for_learning = \
        data_preprocessing.proto_interface.data_for_learning_pb2.ListDataForLearning()
    with open(filepath, 'rb') as file_in:
        list_of_data_for_learning.ParseFromString(file_in.read())
    return list_of_data_for_learning.data_for_learning

In [3]:
data_for_learning = LoadDataForLearning('/home/jiacheng/work/apollo/data/2018-10-01/datalearn.0.bin')

In [67]:
data_for_learning[0].features_for_learning

[1.0, -0.0, -0.0, 5.210425763274543, -4.656612873077393e-10, 3.9841264191200025, 0.5686593716964126, 0.0, 0.0, 1.0, -1.0595502900638816, -0.008417255881762543, 0.0, 0.0, 0.0, 0.0, -0.009721116043709621, 0.09716265080947274, 1.0, -2.2223524588050405, 0.2591302837404676, 0.0, 0.0, 0.0, 0.0, -0.020223043813255792, 0.10495936818343855, 1.0, -3.792662686862465, -0.12339640441358335, 0.0, 0.0, 0.0, 0.0, -0.014930240196906475, -0.052901624388203304, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0191138770119093, 0.010683958962643424, 0.0006856137928155093, 0.026976553174221846, 3.019252371191637, 0.21068368953592254, 0.006080951404213053, 0.025193523867161113, 3.0204683563276866, 0.4106799269746241, 0.0010422214372569627, 0.02481718641336673, 3.0206770055368484, 0.6106797350688413, 0.006005683537116724, 0.0010055108708657743, 3.0218778195952645, 0.8106760263404887, 0.0062067867168007496, 0.025276980156562868, 3.023119169134902, 1.0106721737877262, 0.0011513654085080205, 0.023725801604018427,

In [70]:
features = []
features.append(data_for_learning[0].features_for_learning)
features.append(data_for_learning[10].features_for_learning)
features.append(data_for_learning[33].features_for_learning)

In [69]:
len(num_laneseq)

3

In [68]:
padded_features.shape

(3, 2445)

In [195]:
feature_lengths = [len(feature) for feature in features]
max_feature_len = max(feature_lengths)
print ('The max. feature length is {}'.format(max_feature_len))
num_laneseq = [int((len(feature)-45)/400) for feature in features]
N = len(features)
padded_features = np.zeros((N, max_feature_len+1))

for i, feature_len in enumerate(feature_lengths):
    padded_features[i, 0:feature_len] = features[i][0:feature_len]
    padded_features[i, -1] = num_laneseq[i]

The max. feature length is 2445


In [196]:
padded_features.shape

(3, 2446)

In [197]:
class lane_scanning_model(torch.nn.Module):
    def __init__(self):
        super(lane_scanning_model, self).__init__()
        self.single_lane_cnn = torch.nn.Sequential(
            # L=4x100
            nn.Conv1d(4, 10, 4, stride=2),
            nn.ReLU(),
            # L=10x49
            nn.Conv1d(10, 16, 3, stride=2),
            nn.ReLU(),
            # L=16x24
            nn.Conv1d(16, 25, 3, stride=3),
            nn.ReLU(),
            # L=25x8
        )
        self.single_lane_maxpool = nn.MaxPool1d(4)
        self.single_lane_dropout = nn.Dropout(0.0)

        self.multi_lane_rnn = nn.GRU(
            input_size=50,
            hidden_size=34,
            bidirectional=True,
            batch_first=True)
        self.multi_lane_fc = torch.nn.Sequential(
            nn.Linear(68 * 4, 110),
            nn.ReLU(),
            nn.Dropout(0.0),
            nn.Linear(110, 56),
            nn.ReLU(),
            nn.Dropout(0.0),
        )

        self.obs_feature_fc = torch.nn.Sequential(
            nn.Linear(45, 35),
            nn.ReLU(),
            nn.Dropout(0.0),
            nn.Linear(35, 23),
            nn.ReLU(),
            nn.Dropout(0.0),
        )

    def forward(self, x):
        inputs = x[0]       # N x (45 + 400 * num_laneseq)
        lengths = x[1]      # list of N elements

        obs_in = inputs[:, :45]         # N x 45
        lane_in = inputs[:, 45:]        # N x (400 * max_num_laneseq)
        N = lane_in.size(0)
        max_num_laneseq = lane_in.size(1) / 400

        # N x (max_num_laneseq * 100 * 4)
        lane_in = lane_in.view(N * max_num_laneseq, 100, 4)
        lane_in = lane_in.transpose(1, 2)
        # (N * max_num_laneseq) x 4 x 100
        lane_fea = self.single_lane_cnn(lane_in)
        # (N * max_num_laneseq) x 25 x 8
        lane_fea = self.single_lane_maxpool(lane_fea)
        # (N * max_num_laneseq) x 25 x 2
        lane_fea = lane_fea.view(N*max_num_laneseq, -1)
        lane_fea = self.single_lane_dropout(lane_fea)
        lane_fea = lane_fea.view(N, max_num_laneseq, -1)
        # N x max_num_laneseq x 50

        static_fea = pack_padded_sequence(lane_fea, num_laneseq, batch_first=True)
        static_fea_states, _ = model.multi_lane_rnn(static_fea)
        static_fea, _ = pad_packed_sequence(static_fea_states, batch_first=True)
        min_val = torch.min(static_fea)
        static_fea_maxpool, _ = pad_packed_sequence(
            static_fea_states, batch_first=True, padding_value=min_val.item()-0.1)
        static_fea_maxpool, _ = torch.max(static_fea_maxpool, 1)
        static_fea_avgpool = torch.sum(static_fea, 1) / num_laneseq.reshape(N,1)
        static_fea_front = static_fea[:,0]


In [198]:
model = lane_scanning_model()
padded_features_tensor = Variable(torch.FloatTensor(padded_features).cuda())

In [199]:
model.cuda()

lane_scanning_model(
  (single_lane_cnn): Sequential(
    (0): Conv1d(4, 10, kernel_size=(4,), stride=(2,))
    (1): ReLU()
    (2): Conv1d(10, 16, kernel_size=(3,), stride=(2,))
    (3): ReLU()
    (4): Conv1d(16, 25, kernel_size=(3,), stride=(3,))
    (5): ReLU()
  )
  (single_lane_maxpool): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (single_lane_dropout): Dropout(p=0.0)
  (multi_lane_rnn): GRU(50, 34, batch_first=True, bidirectional=True)
  (multi_lane_fc): Sequential(
    (0): Linear(in_features=272, out_features=110, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.0)
    (3): Linear(in_features=110, out_features=56, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.0)
  )
  (obs_feature_fc): Sequential(
    (0): Linear(in_features=45, out_features=35, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.0)
    (3): Linear(in_features=35, out_features=23, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.0)
  )
)

In [200]:
def batch_preprocess(X, y):
    idx_1 = np.arange(X.shape[0]).tolist()
    idx_2 = torch.argsort(X[:,-1], descending=True).tolist()
    #X[idx_1] = X[idx_2]
    #y[idx_1] = y[idx_2]
    return X[idx_2], y[idx_2]

In [201]:
lane_in = padded_features_tensor[:, 45:]
num_laneseq = padded_features_tensor[:, -1]
N = lane_in.size(0)
max_num_laneseq = int(lane_in.size(1) / 400)

In [205]:
lane_in, num_laneseq = batch_preprocess(lane_in, num_laneseq)

In [209]:
padded_features_tensor[:, 45:]

tensor([[ 3.0191e+00,  1.0684e-02,  6.8561e-04,  ...,  5.7791e+01,
          0.0000e+00,  6.0000e+00],
        [ 2.9482e-01,  1.1051e-03, -3.6771e-11,  ...,  0.0000e+00,
          0.0000e+00,  2.0000e+00],
        [ 4.7978e-01,  1.8207e-02,  9.8623e-12,  ...,  0.0000e+00,
          0.0000e+00,  4.0000e+00]], device='cuda:0')

In [210]:
lane_in

tensor([[ 3.0191e+00,  1.0684e-02,  6.8561e-04,  ...,  5.7791e+01,
          0.0000e+00,  6.0000e+00],
        [ 4.7978e-01,  1.8207e-02,  9.8623e-12,  ...,  0.0000e+00,
          0.0000e+00,  4.0000e+00],
        [ 2.9482e-01,  1.1051e-03, -3.6771e-11,  ...,  0.0000e+00,
          0.0000e+00,  2.0000e+00]], device='cuda:0')

In [219]:
num_laneseq.cpu().numpy()

array([6., 4., 2.], dtype=float32)

In [214]:
num_laneseq[0].item()

6.0

In [15]:
lane_in = lane_in.contiguous()
lane_in = lane_in.view(N * max_num_laneseq, 100, 4)
lane_in = lane_in.transpose(1, 2)

In [16]:
lane_fea = model.single_lane_cnn(lane_in)
lane_fea = model.single_lane_maxpool(lane_fea)

In [17]:
lane_fea = lane_fea.view(N*max_num_laneseq, -1)
lane_fea = model.single_lane_dropout(lane_fea)
lane_fea = lane_fea.view(N, max_num_laneseq, -1)

In [47]:
static_fea = pack_padded_sequence(lane_fea, num_laneseq, batch_first=True)
static_fea_states, _ = model.multi_lane_rnn(static_fea)
static_fea, _ = pad_packed_sequence(static_fea_states, batch_first=True)
min_val = torch.min(static_fea)
static_fea_maxpool, _ = pad_packed_sequence(static_fea_states, batch_first=True, padding_value=min_val.item()-0.1)

In [49]:
static_fea_maxpool, _ = torch.max(static_fea_maxpool, 1)
static_fea_avgpool = torch.sum(static_fea, 1) / num_laneseq.reshape(N,1)
static_fea_front = static_fea[:,0]

idx_1 = np.arange(N).tolist()
idx_2 = (num_laneseq-1).int().data.tolist()
static_fea_back = static_fea[idx_1, idx_2]

In [50]:
static_fea_all = torch.cat((static_fea_maxpool, static_fea_avgpool, static_fea_front, static_fea_back), 1)

In [52]:
static_fea_final = model.multi_lane_fc(static_fea_all)

In [54]:
obs_in = padded_features_tensor[:, :45]
obs_fea_final = model.obs_feature_fc(obs_in)

In [57]:
fea_all = torch.cat((obs_fea_final, static_fea_final), 1)

In [58]:
fea_all.shape

torch.Size([3, 79])

In [251]:
static_fea_all.shape

torch.Size([3, 80])

In [43]:
static_fea[2][0]

tensor([ 0.0000,  0.0000,  1.0000, -1.0000, -1.0000,  0.0000,  1.0000,  0.0000,
         1.0000, -1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,
         1.0000, -1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  1.0000,
         0.0000, -1.0000, -1.0000, -1.0000,  0.0000, -1.0000, -1.0000, -1.0000,
        -1.0000,  0.0000, -1.0000,  0.0000, -1.0000,  1.0000, -1.0000,  0.0000,
         1.0000, -1.0000,  0.0000,  0.0000,  0.0000, -1.0000, -0.9214,  1.0000,
         1.0000, -1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.9990,  0.0000, -1.0000,  0.0000, -1.0000, -1.0000,  0.0000,
         1.0000, -1.0000,  1.0000, -1.0000], device='cuda:0',
       grad_fn=<SelectBackward>)

In [39]:
static_fea_back[2]

tensor([ 0.0000,  0.0000,  1.0000, -1.0000, -1.0000,  0.0000,  1.0000,  0.0000,
         1.0000, -1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,
         1.0000, -1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  1.0000,
         0.0000, -1.0000, -1.0000, -1.0000,  0.0000, -1.0000, -1.0000, -1.0000,
        -1.0000,  0.0000, -1.0000,  0.0000, -1.0000,  1.0000, -1.0000,  0.0000,
         1.0000, -1.0000,  0.0000,  0.0000,  0.0000, -1.0000, -0.7250,  1.0000,
         1.0000, -1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.9977,  0.0000, -1.0000,  0.0000, -1.0000, -1.0000,  0.0000,
         1.0000, -1.0000,  1.0000, -1.0000], device='cuda:0',
       grad_fn=<SelectBackward>)